In [ ]:
import pathlib

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import IPython

import librosa
import librosa.display

# our own stuff
import analog

Song is *Reverie* by [\_ghost](http://ccmixter.org/files/_ghost/25389) (downloaded from [ccMixter](http://ccmixter.org/) under [Creative Commons licences](https://creativecommons.org/licenses/by/3.0/)).

# Introduction

Please check the first few [slides of the course](https://manuvazquez.github.io/assets/communications_theory/slides/analog_modulations.pdf) for this module, until *Types of Modulations*. They just review some ideas we saw during the [course introduction](https://manuvazquez.github.io/assets/communications_theory/slides/introduction.pdf) when talking about *Analog vs Digital communications systems*

# Amplitude Modulation (conventional AM)

An amplitude modulation is a kind of *linear* or *amplitude* (**analog**) modulation, i.e., the information signal is embedded in the amplitude of the signal (meaning the frequency and phase of the *carrier signal* stay constant). If we denote the information signal (also referred to as _modulat**ing**_ signal) by $x(t)$, then the _modulat**ed**_ signal is given by

$$
\large
y(t)
=
\left( 
    A_c +
    A_m
    x(t)
\right)
\cos (w_ct)
$$
where
* both $A_c$ and $A_m$ are (adjustable) modulation parameters 
* $w_c$ is the carrier frequency

## Information signal

Let's build a (deterministic) information (*modulating*) signal

In [ ]:
# parameters
signal_duration = 1.0 # seconds
sampling_frequency = 200.0 # Hz
w_c = np.pi * 20 # carrier frequency

n_samples = int(sampling_frequency * signal_duration)
t = np.arange(n_samples) / sampling_frequency

# the information signal
modulating_signal = 0.1*np.cos(2.0*np.pi*10*t) + np.sin(2.0*np.pi*3*t) + 10*(t-t.mean())**2 - 2

The resulting signal is

In [ ]:
y_limits = [-3.25, 1.1]
figure_size = (12,8)

fig, ax = plt.subplots(1, 1, figsize=figure_size)
ax.plot(t, modulating_signal, label='information signal')
ax.set_ylim(y_limits)
ax.legend();

## Normalized information signal

The minimum value of the signal is about $-3$ and the maximum slightly above $0.75$. For reasons that will be clear in a moment, a signal that is to be transmitted using AM must be constrained between $-1$ and $1$. However, this is not a problem in practice: we can always *normalize* the signal by the maximum of its absolute value, i.e.,

$$
    \large
    x_n(t)
    =
    \frac{x(t)}
    {
        \max
        |x(t)|
    }
    ,
$$
to get

In [ ]:
modulating_signal /= np.abs(modulating_signal).max()

fig, ax = plt.subplots(1, 1, figsize=figure_size)
ax.plot(t, modulating_signal, label='information signal')
ax.set_ylim(y_limits)
for y in [-1, 1]:
    ax.axhline(y=y, linestyle='dashed', color='gray')
ax.legend();

Notice that the normalized signal conveys exactly the **same information** (information is in the shape, not the amplitude). Moreover, one could think of this transformation as simple change of units (such as, e.g., going from kilometers to meters).

## Expressing the modulated signal in a different way

We can rewrite the *modulated* signal in a different way

$$
    \large
    y(t)
    =
    \left( 
        A_c +
        \frac{A_c}{A_c}
        A_m
        x(t)
    \right)
    \cos (w_ct)
    =
    A_c
    \left(
        1 + mx(t)
    \right)
    \cos (w_ct)
$$
by defining the **modulation index**

$$
    \large
    m
    =
    \frac{A_m}{A_c}
$$
.

# Modulation

Let's plot again the information (_modulat**ing**_) signal along with the resulted _modulat**ed**_ signal:

In [ ]:
# an object implementing the above equations
am = analog.AmplitudeModulation(Am=0.5, Ac=1., carrier_freq=w_c)

am.mpl_plot_modulation(t, modulating_signal)

# Demodulation

Looking at the above equation, demodulation is very easy if

$$
    \large
    A_c
    \left(
        1 + mx(t)
    \right)
    \ge
    0
$$

i.e., if the term multiplying the cosine is (at every time instant) non-negative. The reason is that whatever *positive* signal multiplies a rapidly varying cosine constitutes its so-called upper [envelope](https://en.wikipedia.org/wiki/Envelope_(waves)) (a smooth signal that outlines the extremes of a sinusoid), and simple/cheap/efficient hardware is available to extract the latter. Now, if the signal is at some point negative, then it cannot be recovered as the envelope of the signal. In our particular case, what do we need for the condition
$
    A_c
    \left(
        1 + mx(t)
    \right)
    \ge
    0
$
to hold? Above we have guaranteed that $|x(t)| \le 1$. Let us also assume that $A_c \ge 0$ (no need to go into details, but this is not a problem). Then,  we just need to choose $m$ so that $mx(t) \ge -1$, i.e., $ 0 < m \le 1$.

So, in summary, if the modulation index, $m$ is between $0$ and $1$, then the envelope of the modulated signal (easy to extract) is exactly
$
    A_c
    \left(
        1 + mx(t)
    \right)
    \ge
    0
$, and from the latter one can solve for $x(t)$ to recover the information signal. If $m>1$, then the envelope of the signal doesn't match anymore
$
    A_c
    \left(
        1 + mx(t)
    \right)
    \ge
    0
$
and the signal recovered with this envelope-based method is not correct. This is called **overmodulation**.

The figure below allows one to experiment with different values for the modulation index. It can be seen that when $m>1$ (overmodulation), the *envelope* is not equal to $A_c(1 + mx(t))$ anymore, and hence neither is the *demodulated signal* to the information one. Notice that the envelope is *always* positive and hence, whenever $A_c(1 + mx(t))$ becomes negative, we get in trouble.

In [ ]:
@interact(m=(0.25,2, 0.25))
def plot(m=0.5):
    
    # the modulation index is changed...
    am.m = m
    
    # and the modulated, envelope signal and "cosine_factor" (the signal in the above equation)
    modulated_signal, envelope, cosine_factor = am.modulate(t, modulating_signal)

    # the signal is demodulated through the (upper) envelope
    demodulated_signal = (analog.upper_envelope(modulated_signal) / am.Ac - 1.) / am.m

    # figure
    fig, ax0 = plt.subplots(1, 1, figsize=(15,8))

    ax0.plot(t, modulated_signal, linestyle='dashed', color='gray', label='modulated signal')
    ax0.plot(t, envelope, label='envelope', marker='.', markevery=5)
    ax0.plot(t, cosine_factor, label='$A_c(1 + mx(t))$', marker='s', markevery=5)
    ax0.plot(t, modulating_signal, label='information (modulating) signal')
    ax0.plot(t, demodulated_signal, label='demodulated signal', marker='P', markevery=5)
    ax0.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left',fontsize='x-large')

In [ ]:
# filename = pathlib.Path('02 Eruption.mp3')
filename = pathlib.Path('_ghost_-_Reverie_(small_theme).mp3')
assert filename.exists()

In [ ]:
signal, sampling_rate = librosa.load(filename)

In [ ]:
signal.shape

In [ ]:
print(f'Sampling rate is {sampling_rate}')

Signal is *normalized*

In [ ]:
# assert np.abs(signal).max() < 1
# if np.abs(signal).max() < 1:
normalized_signal, normalization_const = analog.normalize(signal, return_normalization_constant=True)
normalization_const

In [ ]:
period = 1. / sampling_frequency
period

The time axis

In [ ]:
time = np.arange(len(normalized_signal))*period
time[:10]

In [ ]:
librosa.display.waveplot(normalized_signal);

In [ ]:
IPython.display.Audio(normalized_signal, rate=sampling_rate)

In [ ]:
# interval_of_interest = range(1_900,2_000)
# interval_of_interest = range(2_000,2_500)
interval_of_interest = range(2_000,2_200)

In [ ]:
signal_piece = normalized_signal[interval_of_interest]
time_piece = time[interval_of_interest]

In [ ]:
plt.plot(time_piece, signal_piece);

In [ ]:
librosa.display.waveplot(signal_piece);

In [ ]:
# IPython.display.Audio(signal_piece, rate=sampling_rate)

### Modulation

In [ ]:
modulated_signal, demodulated_signal, player = None, None, None

In [ ]:
common_properties = {'min': 0.1, 'max': 50.}
Am_slider_widget = widgets.FloatSlider(**common_properties, value=1., description='$A_m$')
Ac_slider_widget = widgets.FloatSlider(**common_properties, value=2, description='$A_c$')
ui = widgets.VBox([Am_slider_widget, Ac_slider_widget])

# player = IPython.display.Audio(demodulated_signal, rate=sampling_rate)

def f(Am: float, Ac: float):
    
    global modulated_signal, demodulated_signal, player
    
    am = analog.AmplitudeModulation(Am=Am, Ac=Ac, carrier_freq=1e5)
    modulated_signal, envelope, cosine_factor = am.modulate(time, normalized_signal)
    demodulated_signal = am.demodulate(modulated_signal)
    
    # figure
    fig, ax = plt.subplots(1, 1, figsize=(15,8))

#     ax.plot(
#         time[interval_of_interest], modulated_signal[interval_of_interest],
#         linestyle='dashed', color='gray', label='modulated signal')
#     ax0.plot(t, envelope, label='envelope', marker='.', markevery=5)
#     ax0.plot(t, cosine_factor, label='$A_c(1 + mx(t))$', marker='s', markevery=5)
    ax.plot(
        time[interval_of_interest], normalized_signal[interval_of_interest], label='information (modulating) signal')
    ax.plot(
        time[interval_of_interest], demodulated_signal[interval_of_interest], label='demodulated signal',
        marker='P', markevery=5)
    ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left',fontsize='x-large')
    player = IPython.display.Audio(demodulated_signal, rate=sampling_rate)
#     player.reload()
    print(f'm = {am.m}')

out = widgets.interactive_output(f, {'Am': Am_slider_widget, 'Ac': Ac_slider_widget})
# IPython.display.display(ui, out)
IPython.display.display(ui, out, player)

In [ ]:
player.reload()
dir(player)

In [ ]:
player

In [ ]:
# IPython.display.Audio(demodulated_signal, rate=sampling_rate)

* Try different values of the parameters.
* Does overmodulation damage the signal? What is the required degree to actually be able to *hear the difference*?